# Preparation for Colab

Make sure you're running a GPU runtime; if not, select "GPU" as the hardware accelerator in Runtime > Change Runtime Type in the menu. The next cells will install the `clip` package and its dependencies, and check if PyTorch 1.7.1 or later is installed.

In [1]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ftfy]1/3 [regex]
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-al9q4ci9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-al9q4ci9
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 95.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 140.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 145.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 146.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [1]:
import numpy as np
import torch
import clip
from tqdm.notebook import tqdm
from pkg_resources import packaging

print("Torch version:", torch.__version__)


Torch version: 2.7.1+cu126


/tmp/ipykernel_3746513/926775896.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging


# Loading the model

Download and instantiate a CLIP model using the `clip` module that we just installed.

In [2]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [3]:
model, preprocess = clip.load("ViT-B/32")

In [4]:
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


# Preparing ImageNet labels and prompts

The following cell contains the 1,000 labels for the ImageNet dataset, followed by the text templates we'll use as "prompt engineering".

In [2]:
imagenet_classes = [ "tench", "goldfish", "great white shark", "tiger shark", "hammerhead shark", "electric ray",
    "stingray", "rooster", "hen", "ostrich", "brambling", "goldfinch", "house finch", "junco",
    "indigo bunting", "American robin", "bulbul", "jay", "magpie", "chickadee", "American dipper",
    "kite (bird of prey)", "bald eagle", "vulture", "great grey owl", "fire salamander",
    "smooth newt", "newt", "spotted salamander", "axolotl", "American bullfrog", "tree frog",
    "tailed frog", "loggerhead sea turtle", "leatherback sea turtle", "mud turtle", "terrapin",
    "box turtle", "banded gecko", "green iguana", "Carolina anole",
    "desert grassland whiptail lizard", "agama", "frilled-necked lizard", "alligator lizard",
    "Gila monster", "European green lizard", "chameleon", "Komodo dragon", "Nile crocodile",
    "American alligator", "triceratops", "worm snake", "ring-necked snake",
    "eastern hog-nosed snake", "smooth green snake", "kingsnake", "garter snake", "water snake",
    "vine snake", "night snake", "boa constrictor", "African rock python", "Indian cobra",
    "green mamba", "sea snake", "Saharan horned viper", "eastern diamondback rattlesnake",
    "sidewinder rattlesnake", "trilobite", "harvestman", "scorpion", "yellow garden spider",
    "barn spider", "European garden spider", "southern black widow", "tarantula", "wolf spider",
    "tick", "centipede", "black grouse", "ptarmigan", "ruffed grouse", "prairie grouse", "peafowl",
    "quail", "partridge", "african grey parrot", "macaw", "sulphur-crested cockatoo", "lorikeet",
    "coucal", "bee eater", "hornbill", "hummingbird", "jacamar", "toucan", "duck",
    "red-breasted merganser", "goose", "black swan", "tusker", "echidna", "platypus", "wallaby",
    "koala", "wombat", "jellyfish", "sea anemone", "brain coral", "flatworm", "nematode", "conch",
    "snail", "slug", "sea slug", "chiton", "chambered nautilus", "Dungeness crab", "rock crab",
    "fiddler crab", "red king crab", "American lobster", "spiny lobster", "crayfish", "hermit crab",
    "isopod", "white stork", "black stork", "spoonbill", "flamingo", "little blue heron",
    "great egret", "bittern bird", "crane bird", "limpkin", "common gallinule", "American coot",
    "bustard", "ruddy turnstone", "dunlin", "common redshank", "dowitcher", "oystercatcher",
    "pelican", "king penguin", "albatross", "grey whale", "killer whale", "dugong", "sea lion",
    "Chihuahua", "Japanese Chin", "Maltese", "Pekingese", "Shih Tzu", "King Charles Spaniel",
    "Papillon", "toy terrier", "Rhodesian Ridgeback", "Afghan Hound", "Basset Hound", "Beagle",
    "Bloodhound", "Bluetick Coonhound", "Black and Tan Coonhound", "Treeing Walker Coonhound",
    "English foxhound", "Redbone Coonhound", "borzoi", "Irish Wolfhound", "Italian Greyhound",
    "Whippet", "Ibizan Hound", "Norwegian Elkhound", "Otterhound", "Saluki", "Scottish Deerhound",
    "Weimaraner", "Staffordshire Bull Terrier", "American Staffordshire Terrier",
    "Bedlington Terrier", "Border Terrier", "Kerry Blue Terrier", "Irish Terrier",
    "Norfolk Terrier", "Norwich Terrier", "Yorkshire Terrier", "Wire Fox Terrier",
    "Lakeland Terrier", "Sealyham Terrier", "Airedale Terrier", "Cairn Terrier",
    "Australian Terrier", "Dandie Dinmont Terrier", "Boston Terrier", "Miniature Schnauzer",
    "Giant Schnauzer", "Standard Schnauzer", "Scottish Terrier", "Tibetan Terrier",
    "Australian Silky Terrier", "Soft-coated Wheaten Terrier", "West Highland White Terrier",
    "Lhasa Apso", "Flat-Coated Retriever", "Curly-coated Retriever", "Golden Retriever",
    "Labrador Retriever", "Chesapeake Bay Retriever", "German Shorthaired Pointer", "Vizsla",
    "English Setter", "Irish Setter", "Gordon Setter", "Brittany dog", "Clumber Spaniel",
    "English Springer Spaniel", "Welsh Springer Spaniel", "Cocker Spaniel", "Sussex Spaniel",
    "Irish Water Spaniel", "Kuvasz", "Schipperke", "Groenendael dog", "Malinois", "Briard",
    "Australian Kelpie", "Komondor", "Old English Sheepdog", "Shetland Sheepdog", "collie",
    "Border Collie", "Bouvier des Flandres dog", "Rottweiler", "German Shepherd Dog", "Dobermann",
    "Miniature Pinscher", "Greater Swiss Mountain Dog", "Bernese Mountain Dog",
    "Appenzeller Sennenhund", "Entlebucher Sennenhund", "Boxer", "Bullmastiff", "Tibetan Mastiff",
    "French Bulldog", "Great Dane", "St. Bernard", "husky", "Alaskan Malamute", "Siberian Husky",
    "Dalmatian", "Affenpinscher", "Basenji", "pug", "Leonberger", "Newfoundland dog",
    "Great Pyrenees dog", "Samoyed", "Pomeranian", "Chow Chow", "Keeshond", "brussels griffon",
    "Pembroke Welsh Corgi", "Cardigan Welsh Corgi", "Toy Poodle", "Miniature Poodle",
    "Standard Poodle", "Mexican hairless dog (xoloitzcuintli)", "grey wolf", "Alaskan tundra wolf",
    "red wolf or maned wolf", "coyote", "dingo", "dhole", "African wild dog", "hyena", "red fox",
    "kit fox", "Arctic fox", "grey fox", "tabby cat", "tiger cat", "Persian cat", "Siamese cat",
    "Egyptian Mau", "cougar", "lynx", "leopard", "snow leopard", "jaguar", "lion", "tiger",
    "cheetah", "brown bear", "American black bear", "polar bear", "sloth bear", "mongoose",
    "meerkat", "tiger beetle", "ladybug", "ground beetle", "longhorn beetle", "leaf beetle",
    "dung beetle", "rhinoceros beetle", "weevil", "fly", "bee", "ant", "grasshopper",
    "cricket insect", "stick insect", "cockroach", "praying mantis", "cicada", "leafhopper",
    "lacewing", "dragonfly", "damselfly", "red admiral butterfly", "ringlet butterfly",
    "monarch butterfly", "small white butterfly", "sulphur butterfly", "gossamer-winged butterfly",
    "starfish", "sea urchin", "sea cucumber", "cottontail rabbit", "hare", "Angora rabbit",
    "hamster", "porcupine", "fox squirrel", "marmot", "beaver", "guinea pig", "common sorrel horse",
    "zebra", "pig", "wild boar", "warthog", "hippopotamus", "ox", "water buffalo", "bison",
    "ram (adult male sheep)", "bighorn sheep", "Alpine ibex", "hartebeest", "impala (antelope)",
    "gazelle", "arabian camel", "llama", "weasel", "mink", "European polecat",
    "black-footed ferret", "otter", "skunk", "badger", "armadillo", "three-toed sloth", "orangutan",
    "gorilla", "chimpanzee", "gibbon", "siamang", "guenon", "patas monkey", "baboon", "macaque",
    "langur", "black-and-white colobus", "proboscis monkey", "marmoset", "white-headed capuchin",
    "howler monkey", "titi monkey", "Geoffroy's spider monkey", "common squirrel monkey",
    "ring-tailed lemur", "indri", "Asian elephant", "African bush elephant", "red panda",
    "giant panda", "snoek fish", "eel", "silver salmon", "rock beauty fish", "clownfish",
    "sturgeon", "gar fish", "lionfish", "pufferfish", "abacus", "abaya", "academic gown",
    "accordion", "acoustic guitar", "aircraft carrier", "airliner", "airship", "altar", "ambulance",
    "amphibious vehicle", "analog clock", "apiary", "apron", "trash can", "assault rifle",
    "backpack", "bakery", "balance beam", "balloon", "ballpoint pen", "Band-Aid", "banjo",
    "baluster / handrail", "barbell", "barber chair", "barbershop", "barn", "barometer", "barrel",
    "wheelbarrow", "baseball", "basketball", "bassinet", "bassoon", "swimming cap", "bath towel",
    "bathtub", "station wagon", "lighthouse", "beaker", "military hat (bearskin or shako)",
    "beer bottle", "beer glass", "bell tower", "baby bib", "tandem bicycle", "bikini",
    "ring binder", "binoculars", "birdhouse", "boathouse", "bobsleigh", "bolo tie", "poke bonnet",
    "bookcase", "bookstore", "bottle cap", "hunting bow", "bow tie", "brass memorial plaque", "bra",
    "breakwater", "breastplate", "broom", "bucket", "buckle", "bulletproof vest",
    "high-speed train", "butcher shop", "taxicab", "cauldron", "candle", "cannon", "canoe",
    "can opener", "cardigan", "car mirror", "carousel", "tool kit", "cardboard box / carton",
    "car wheel", "automated teller machine", "cassette", "cassette player", "castle", "catamaran",
    "CD player", "cello", "mobile phone", "chain", "chain-link fence", "chain mail", "chainsaw",
    "storage chest", "chiffonier", "bell or wind chime", "china cabinet", "Christmas stocking",
    "church", "movie theater", "cleaver", "cliff dwelling", "cloak", "clogs", "cocktail shaker",
    "coffee mug", "coffeemaker", "spiral or coil", "combination lock", "computer keyboard",
    "candy store", "container ship", "convertible", "corkscrew", "cornet", "cowboy boot",
    "cowboy hat", "cradle", "construction crane", "crash helmet", "crate", "infant bed",
    "Crock Pot", "croquet ball", "crutch", "cuirass", "dam", "desk", "desktop computer",
    "rotary dial telephone", "diaper", "digital clock", "digital watch", "dining table",
    "dishcloth", "dishwasher", "disc brake", "dock", "dog sled", "dome", "doormat", "drilling rig",
    "drum", "drumstick", "dumbbell", "Dutch oven", "electric fan", "electric guitar",
    "electric locomotive", "entertainment center", "envelope", "espresso machine", "face powder",
    "feather boa", "filing cabinet", "fireboat", "fire truck", "fire screen", "flagpole", "flute",
    "folding chair", "football helmet", "forklift", "fountain", "fountain pen", "four-poster bed",
    "freight car", "French horn", "frying pan", "fur coat", "garbage truck",
    "gas mask or respirator", "gas pump", "goblet", "go-kart", "golf ball", "golf cart", "gondola",
    "gong", "gown", "grand piano", "greenhouse", "radiator grille", "grocery store", "guillotine",
    "hair clip", "hair spray", "half-track", "hammer", "hamper", "hair dryer", "hand-held computer",
    "handkerchief", "hard disk drive", "harmonica", "harp", "combine harvester", "hatchet",
    "holster", "home theater", "honeycomb", "hook", "hoop skirt", "gymnastic horizontal bar",
    "horse-drawn vehicle", "hourglass", "iPod", "clothes iron", "carved pumpkin", "jeans", "jeep",
    "T-shirt", "jigsaw puzzle", "rickshaw", "joystick", "kimono", "knee pad", "knot", "lab coat",
    "ladle", "lampshade", "laptop computer", "lawn mower", "lens cap", "letter opener", "library",
    "lifeboat", "lighter", "limousine", "ocean liner", "lipstick", "slip-on shoe", "lotion",
    "music speaker", "loupe magnifying glass", "sawmill", "magnetic compass", "messenger bag",
    "mailbox", "tights", "one-piece bathing suit", "manhole cover", "maraca", "marimba", "mask",
    "matchstick", "maypole", "maze", "measuring cup", "medicine cabinet", "megalith", "microphone",
    "microwave oven", "military uniform", "milk can", "minibus", "miniskirt", "minivan", "missile",
    "mitten", "mixing bowl", "mobile home", "ford model t", "modem", "monastery", "monitor",
    "moped", "mortar and pestle", "graduation cap", "mosque", "mosquito net", "vespa",
    "mountain bike", "tent", "computer mouse", "mousetrap", "moving van", "muzzle", "metal nail",
    "neck brace", "necklace", "baby pacifier", "notebook computer", "obelisk", "oboe", "ocarina",
    "odometer", "oil filter", "pipe organ", "oscilloscope", "overskirt", "bullock cart",
    "oxygen mask", "product packet / packaging", "paddle", "paddle wheel", "padlock", "paintbrush",
    "pajamas", "palace", "pan flute", "paper towel", "parachute", "parallel bars", "park bench",
    "parking meter", "railroad car", "patio", "payphone", "pedestal", "pencil case",
    "pencil sharpener", "perfume", "Petri dish", "photocopier", "plectrum", "Pickelhaube",
    "picket fence", "pickup truck", "pier", "piggy bank", "pill bottle", "pillow", "ping-pong ball",
    "pinwheel", "pirate ship", "drink pitcher", "block plane", "planetarium", "plastic bag",
    "plate rack", "farm plow", "plunger", "Polaroid camera", "pole", "police van", "poncho",
    "pool table", "soda bottle", "plant pot", "potter's wheel", "power drill", "prayer rug",
    "printer", "prison", "missile", "projector", "hockey puck", "punching bag", "purse", "quill",
    "quilt", "race car", "racket", "radiator", "radio", "radio telescope", "rain barrel",
    "recreational vehicle", "fishing casting reel", "reflex camera", "refrigerator",
    "remote control", "restaurant", "revolver", "rifle", "rocking chair", "rotisserie", "eraser",
    "rugby ball", "ruler measuring stick", "sneaker", "safe", "safety pin", "salt shaker", "sandal",
    "sarong", "saxophone", "scabbard", "weighing scale", "school bus", "schooner", "scoreboard",
    "CRT monitor", "screw", "screwdriver", "seat belt", "sewing machine", "shield", "shoe store",
    "shoji screen / room divider", "shopping basket", "shopping cart", "shovel", "shower cap",
    "shower curtain", "ski", "balaclava ski mask", "sleeping bag", "slide rule", "sliding door",
    "slot machine", "snorkel", "snowmobile", "snowplow", "soap dispenser", "soccer ball", "sock",
    "solar thermal collector", "sombrero", "soup bowl", "keyboard space bar", "space heater",
    "space shuttle", "spatula", "motorboat", "spider web", "spindle", "sports car", "spotlight",
    "stage", "steam locomotive", "through arch bridge", "steel drum", "stethoscope", "scarf",
    "stone wall", "stopwatch", "stove", "strainer", "tram", "stretcher", "couch", "stupa",
    "submarine", "suit", "sundial", "sunglasses", "sunglasses", "sunscreen", "suspension bridge",
    "mop", "sweatshirt", "swim trunks / shorts", "swing", "electrical switch", "syringe",
    "table lamp", "tank", "tape player", "teapot", "teddy bear", "television", "tennis ball",
    "thatched roof", "front curtain", "thimble", "threshing machine", "throne", "tile roof",
    "toaster", "tobacco shop", "toilet seat", "torch", "totem pole", "tow truck", "toy store",
    "tractor", "semi-trailer truck", "tray", "trench coat", "tricycle", "trimaran", "tripod",
    "triumphal arch", "trolleybus", "trombone", "hot tub", "turnstile", "typewriter keyboard",
    "umbrella", "unicycle", "upright piano", "vacuum cleaner", "vase", "vaulted or arched ceiling",
    "velvet fabric", "vending machine", "vestment", "viaduct", "violin", "volleyball",
    "waffle iron", "wall clock", "wallet", "wardrobe", "military aircraft", "sink",
    "washing machine", "water bottle", "water jug", "water tower", "whiskey jug", "whistle",
    "hair wig", "window screen", "window shade", "Windsor tie", "wine bottle", "airplane wing",
    "wok", "wooden spoon", "wool", "split-rail fence", "shipwreck", "sailboat", "yurt", "website",
    "comic book", "crossword", "traffic or street sign", "traffic light", "dust jacket", "menu",
    "plate", "guacamole", "consomme", "hot pot", "trifle", "ice cream", "popsicle", "baguette",
    "bagel", "pretzel", "cheeseburger", "hot dog", "mashed potatoes", "cabbage", "broccoli",
    "cauliflower", "zucchini", "spaghetti squash", "acorn squash", "butternut squash", "cucumber",
    "artichoke", "bell pepper", "cardoon", "mushroom", "Granny Smith apple", "strawberry", "orange",
    "lemon", "fig", "pineapple", "banana", "jackfruit", "cherimoya (custard apple)", "pomegranate",
    "hay", "carbonara", "chocolate syrup", "dough", "meatloaf", "pizza", "pot pie", "burrito",
    "red wine", "espresso", "tea cup", "eggnog", "mountain", "bubble", "cliff", "coral reef",
    "geyser", "lakeshore", "promontory", "sandbar", "beach", "valley", "volcano", "baseball player",
    "bridegroom", "scuba diver", "rapeseed", "daisy", "yellow lady's slipper", "corn", "acorn",
    "rose hip", "horse chestnut seed", "coral fungus", "agaric", "gyromitra", "stinkhorn mushroom",
    "earth star fungus", "hen of the woods mushroom", "bolete", "corn cob", "toilet paper"
]

A subset of these class names are modified from the default ImageNet class names sourced from Anish Athalye's imagenet-simple-labels.

These edits were made via trial and error and concentrated on the lowest performing classes according to top_1 and top_5 accuracy on the ImageNet training set for the RN50, RN101, and RN50x4 models. These tweaks improve top_1 by 1.5% on ViT-B/32 over using the default class names. Alec got bored somewhere along the way as gains started to diminish and never finished updating / tweaking the list. He also didn't revisit this with the better performing RN50x16, RN50x64, or any of the ViT models. He thinks it's likely another 0.5% to 1% top_1 could be gained from further work here. It'd be interesting to more rigorously study / understand this.

Some examples beyond the crane/crane -> construction crane / bird crane issue mentioned in Section 3.1.4 of the paper include:

- CLIP interprets "nail" as "fingernail" so we changed the label to "metal nail".
- ImageNet kite class refers to the bird of prey, not the flying toy, so we changed "kite" to "kite (bird of prey)"
- The ImageNet class for red wolf seems to include a lot of mislabeled maned wolfs so we changed "red wolf" to "red wolf or maned wolf"

In [6]:
imagenet_templates = [
    'a bad photo of a {}.',
    'a photo of many {}.',
    'a sculpture of a {}.',
    'a photo of the hard to see {}.',
    'a low resolution photo of the {}.',
    'a rendering of a {}.',
    'graffiti of a {}.',
    'a bad photo of the {}.',
    'a cropped photo of the {}.',
    'a tattoo of a {}.',
    'the embroidered {}.',
    'a photo of a hard to see {}.',
    'a bright photo of a {}.',
    'a photo of a clean {}.',
    'a photo of a dirty {}.',
    'a dark photo of the {}.',
    'a drawing of a {}.',
    'a photo of my {}.',
    'the plastic {}.',
    'a photo of the cool {}.',
    'a close-up photo of a {}.',
    'a black and white photo of the {}.',
    'a painting of the {}.',
    'a painting of a {}.',
    'a pixelated photo of the {}.',
    'a sculpture of the {}.',
    'a bright photo of the {}.',
    'a cropped photo of a {}.',
    'a plastic {}.',
    'a photo of the dirty {}.',
    'a jpeg corrupted photo of a {}.',
    'a blurry photo of the {}.',
    'a photo of the {}.',
    'a good photo of the {}.',
    'a rendering of the {}.',
    'a {} in a video game.',
    'a photo of one {}.',
    'a doodle of a {}.',
    'a close-up photo of the {}.',
    'a photo of a {}.',
    'the origami {}.',
    'the {} in a video game.',
    'a sketch of a {}.',
    'a doodle of the {}.',
    'a origami {}.',
    'a low resolution photo of a {}.',
    'the toy {}.',
    'a rendition of the {}.',
    'a photo of the clean {}.',
    'a photo of a large {}.',
    'a rendition of a {}.',
    'a photo of a nice {}.',
    'a photo of a weird {}.',
    'a blurry photo of a {}.',
    'a cartoon {}.',
    'art of a {}.',
    'a sketch of the {}.',
    'a embroidered {}.',
    'a pixelated photo of a {}.',
    'itap of the {}.',
    'a jpeg corrupted photo of the {}.',
    'a good photo of a {}.',
    'a plushie {}.',
    'a photo of the nice {}.',
    'a photo of the small {}.',
    'a photo of the weird {}.',
    'the cartoon {}.',
    'art of the {}.',
    'a drawing of the {}.',
    'a photo of the large {}.',
    'a black and white photo of a {}.',
    'the plushie {}.',
    'a dark photo of a {}.',
    'itap of a {}.',
    'graffiti of the {}.',
    'a toy {}.',
    'itap of my {}.',
    'a photo of a cool {}.',
    'a photo of a small {}.',
    'a tattoo of the {}.',
]

print(f"{len(imagenet_classes)} classes, {len(imagenet_templates)} templates")

1000 classes, 80 templates


A similar, intuition-guided trial and error based on the ImageNet training set was used for templates. This list is pretty haphazard and was gradually made / expanded over the course of about a year of the project and was revisited / tweaked every few months. A surprising / weird thing was adding templates intended to help ImageNet-R performance (specifying different possible renditions of an object) improved standard ImageNet accuracy too.

After the 80 templates were "locked" for the paper, we ran sequential forward selection over the list of 80 templates. The search terminated after ensembling 7 templates and selected them in the order below.

1. itap of a {}.
2. a bad photo of the {}.
3. a origami {}.
4. a photo of the large {}.
5. a {} in a video game.
6. art of the {}.
7. a photo of the small {}.

Speculating, we think it's interesting to see different scales (large and small), a difficult view (a bad photo), and "abstract" versions (origami, video game, art), were all selected for, but we haven't studied this in any detail. This subset performs a bit better than the full 80 ensemble reported in the paper, especially for the smaller models.

# Loading the Images

The ILSVRC2012 datasets are no longer available for download publicly. We instead download the ImageNet-V2 dataset by [Recht et al.](https://arxiv.org/abs/1902.10811).

If you have the ImageNet dataset downloaded, you can replace the dataset with the official torchvision loader, e.g.:

```python
images = torchvision.datasets.ImageNet("path/to/imagenet", split='val', transform=preprocess)
```

In [7]:
! pip install git+https://github.com/modestyachts/ImageNetV2_pytorch

from imagenetv2_pytorch import ImageNetV2Dataset

images = ImageNetV2Dataset(transform=preprocess)
loader = torch.utils.data.DataLoader(images, batch_size=32, num_workers=2)

  Cloning https://github.com/modestyachts/ImageNetV2_pytorch to /tmp/pip-req-build-cxywdek7
  Running command git clone --filter=blob:none --quiet https://github.com/modestyachts/ImageNetV2_pytorch /tmp/pip-req-build-cxywdek7
  Resolved https://github.com/modestyachts/ImageNetV2_pytorch to commit 14d4456c39fe7f02a665544dd9fc37c1a5f8b635
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [14]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ipywidgets]2 [ipywidgets]


In [19]:
!jupyter nbextension enable --py widgetsnbextension


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot

Jupyter command `jupyter-nbextension` not found.


# Creating zero-shot classifier weights

In [8]:
def zeroshot_classifier(classnames, templates):
    with torch.no_grad():
        zeroshot_weights = []
        for classname in tqdm(classnames):
            texts = [template.format(classname) for template in templates] #format with class
            texts = clip.tokenize(texts).cuda() #tokenize
            class_embeddings = model.encode_text(texts) #embed with text encoder
            class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
            class_embedding = class_embeddings.mean(dim=0)
            class_embedding /= class_embedding.norm()
            zeroshot_weights.append(class_embedding)
        zeroshot_weights = torch.stack(zeroshot_weights, dim=1).cuda()
    return zeroshot_weights


zeroshot_weights = zeroshot_classifier(imagenet_classes, imagenet_templates)

  0%|          | 0/1000 [00:00<?, ?it/s]

# Zero-shot prediction

In [9]:
def accuracy(output, target, topk=(1,)):
    pred = output.topk(max(topk), 1, True, True)[1].t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    return [float(correct[:k].reshape(-1).float().sum(0, keepdim=True).cpu().numpy()) for k in topk]

In [10]:
with torch.no_grad():
    top1, top5, n = 0., 0., 0.
    for i, (images, target) in enumerate(tqdm(loader)):
        images = images.cuda()
        target = target.cuda()
        
        # predict
        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        logits = 100. * image_features @ zeroshot_weights

        # measure accuracy
        acc1, acc5 = accuracy(logits, target, topk=(1, 5))
        top1 += acc1
        top5 += acc5
        n += images.size(0)

top1 = (top1 / n) * 100
top5 = (top5 / n) * 100 

print(f"Top-1 accuracy: {top1:.2f}")
print(f"Top-5 accuracy: {top5:.2f}")

  0%|          | 0/313 [00:00<?, ?it/s]

/tmp/ipykernel_3746513/2288536226.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return [float(correct[:k].reshape(-1).float().sum(0, keepdim=True).cpu().numpy()) for k in topk]


Top-1 accuracy: 55.90
Top-5 accuracy: 83.42


In [12]:
!pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 54.9 MB/s eta 0:00:00a 0:00:01


In [5]:
pip install wget

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9685 sha256=665b98b188758cd085ba5ebdc7d3ce81dccec5f1c3c464e5f5eacb146b88f9bc
  Stored in directory: /home/infres/pmbathe-24/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import requests
import tarfile
import zipfile
import json
from pathlib import Path
import shutil
import wget
from PIL import Image
import pandas as pd
from tqdm import tqdm
import concurrent.futures
import time

class ImageNetDownloader:
    def __init__(self):
        self.base_dir = Path("imagenet_full")
        self.base_dir.mkdir(exist_ok=True)
        
        # URLs officielles ImageNet (nécessitent inscription)
        self.imagenet_urls = {
            "train": "https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_train.tar",
            "val": "https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar",
            "test": "https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_test_v10102019.tar"
        }
        
        # URLs alternatives (Academic Torrents - plus fiables)
        self.academic_torrents_urls = {
            "train": "https://academictorrents.com/download/a306397ccf9c2ead27155983c254227c0fd938e2.torrent",
            "val": "https://academictorrents.com/download/5d6d0df7ed81efd49ca99ea4737e0ae5e3a5f2e5.torrent"
        }

    def download_imagenet_labels(self):
        """Télécharge les labels et métadonnées ImageNet"""
        print("📋 Téléchargement des labels ImageNet...")
        
        # Labels des classes
        labels_url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
        if not os.path.exists("imagenet_classes.txt"):
            response = requests.get(labels_url)
            with open("imagenet_classes.txt", "w") as f:
                f.write(response.text)
        
        # Mapping WordNet ID -> nom de classe
        synsets_url = "https://raw.githubusercontent.com/HoldenCaulfieldRye/caffe/master/data/ilsvrc12/synsets.txt" 
        if not os.path.exists("synsets.txt"):
            try:
                response = requests.get(synsets_url)
                with open("synsets.txt", "w") as f:
                    f.write(response.text)
            except:
                # Créer manuellement si l'URL ne marche pas
                self.create_synsets_file()
        
        # Charger les labels
        with open("imagenet_classes.txt", "r") as f:
            class_names = [line.strip() for line in f.readlines()]
        
        print(f"✓ {len(class_names)} classes chargées")
        return class_names

    def create_synsets_file(self):
        """Crée le fichier synsets.txt avec les WordNet IDs"""
        # Liste des 1000 classes ImageNet avec leurs WordNet IDs
        # Ici une version simplifiée - normalement il faut les vrais IDs
        synsets = []
        for i in range(1000):
            synsets.append(f"n{i+1:08d}\n")
        
        with open("synsets.txt", "w") as f:
            f.writelines(synsets)

    def download_via_kaggle(self):
        """
        Télécharge ImageNet via Kaggle (plus simple et fiable)
        Nécessite: pip install kaggle et configuration API
        """
        try:
            import kaggle
            print("🔄 Téléchargement via Kaggle...")
            
            # Dataset ImageNet sur Kaggle
            kaggle.api.dataset_download_files(
                "c/imagenet-object-localization-challenge",
                path=str(self.base_dir),
                unzip=True
            )
            print("✓ ImageNet téléchargé via Kaggle")
            return True
            
        except Exception as e:
            print(f"❌ Erreur Kaggle: {e}")
            return False

    def download_official_imagenet(self, subset="val"):
        """
        Télécharge ImageNet depuis le site officiel
        ATTENTION: Nécessite un compte et acceptation des termes
        """
        print(f"🔄 Téléchargement ImageNet {subset} (site officiel)...")
        print("⚠️  IMPORTANT: Vous devez:")
        print("   1. Créer un compte sur image-net.org")
        print("   2. Accepter les termes d'utilisation")
        print("   3. Obtenir l'autorisation de téléchargement")
        
        url = self.imagenet_urls[subset]
        filename = f"ILSVRC2012_img_{subset}.tar"
        
        if not os.path.exists(filename):
            print(f"Tentative de téléchargement de {filename}...")
            print("Si cela échoue, téléchargez manuellement depuis image-net.org")
            
            try:
                # Téléchargement avec wget (plus robuste pour gros fichiers)
                os.system(f"wget -c {url} -O {filename}")
                
                if not os.path.exists(filename) or os.path.getsize(filename) < 1000:
                    print("❌ Téléchargement échoué - fichier manquant ou corrompu")
                    return False
                    
            except Exception as e:
                print(f"❌ Erreur: {e}")
                return False
        
        return True

    def extract_and_organize_train_set(self, tar_path):
        """
        Extrait et organise le training set ImageNet
        Le training set est déjà organisé par classe (n01440764/, n01443537/, etc.)
        """
        print("📦 Extraction du training set...")
        
        output_dir = self.base_dir / "train"
        output_dir.mkdir(exist_ok=True)
        
        try:
            with tarfile.open(tar_path, "r") as tar:
                # Extraire dans un dossier temporaire
                temp_dir = "temp_train_extract"
                tar.extractall(temp_dir)
                
                # Le training set contient des tar individuels pour chaque classe
                class_tars = list(Path(temp_dir).glob("*.tar"))
                print(f"Trouvé {len(class_tars)} classes à extraire")
                
                for class_tar in tqdm(class_tars, desc="Extraction classes"):
                    class_name = class_tar.stem  # ex: n01440764
                    class_dir = output_dir / class_name
                    class_dir.mkdir(exist_ok=True)
                    
                    # Extraire les images de cette classe
                    with tarfile.open(class_tar, "r") as class_tar_file:
                        class_tar_file.extractall(class_dir)
                
                # Nettoyer
                shutil.rmtree(temp_dir)
                print("✓ Training set extrait et organisé")
                return True
                
        except Exception as e:
            print(f"❌ Erreur extraction: {e}")
            return False

    def extract_and_organize_val_set(self, tar_path):
        """
        Extrait et organise le validation set ImageNet
        Le validation set nécessite le fichier de labels pour l'organisation
        """
        print("📦 Extraction du validation set...")
        
        # Télécharger les labels de validation
        val_labels_url = "https://raw.githubusercontent.com/soumith/imagenetloader.torch/master/valprep.sh"
        
        output_dir = self.base_dir / "val"
        output_dir.mkdir(exist_ok=True)
        
        try:
            # Extraire toutes les images
            with tarfile.open(tar_path, "r") as tar:
                tar.extractall(output_dir)
            
            # Télécharger le script de réorganisation
            if not os.path.exists("valprep.sh"):
                response = requests.get(val_labels_url)
                with open("valprep.sh", "w") as f:
                    f.write(response.text)
            
            # Exécuter le script de réorganisation
            os.system(f"chmod +x valprep.sh")
            os.system(f"cd {output_dir} && ../../valprep.sh")
            
            print("✓ Validation set extrait et organisé")
            return True
            
        except Exception as e:
            print(f"❌ Erreur extraction validation: {e}")
            return False

    def create_alternative_download_script(self):
        """
        Crée un script bash pour téléchargement via Academic Torrents
        """
        script_content = """#!/bin/bash

# Script de téléchargement ImageNet via Academic Torrents
# Plus fiable que le site officiel

echo "=== TÉLÉCHARGEMENT IMAGENET VIA ACADEMIC TORRENTS ==="
echo "Installation des dépendances..."

# Installer aria2 (client torrent)
if ! command -v aria2c &> /dev/null; then
    echo "Installation d'aria2..."
    # Ubuntu/Debian
    sudo apt-get update && sudo apt-get install -y aria2
    # MacOS
    # brew install aria2
fi

echo "Téléchargement des fichiers torrent..."

# Télécharger les torrents
wget -O imagenet_train.torrent "https://academictorrents.com/download/a306397ccf9c2ead27155983c254227c0fd938e2.torrent"
wget -O imagenet_val.torrent "https://academictorrents.com/download/5d6d0df7ed81efd49ca99ea4737e0ae5e3a5f2e5.torrent"

echo "Démarrage du téléchargement (peut prendre plusieurs heures)..."

# Télécharger via torrent
aria2c --seed-time=0 imagenet_train.torrent
aria2c --seed-time=0 imagenet_val.torrent

echo "Téléchargement terminé!"
echo "Exécutez ensuite le script Python pour organiser les fichiers."
"""
        
        with open("download_imagenet_torrents.sh", "w") as f:
            f.write(script_content)
        
        os.chmod("download_imagenet_torrents.sh", 0o755)
        print("✓ Script de téléchargement torrent créé: download_imagenet_torrents.sh")

    def download_sample_dataset(self, num_classes=100, images_per_class=50):
        """
        Télécharge un échantillon d'ImageNet pour tester
        Utilise l'API Flickr pour obtenir des images similaires
        """
        print(f"🔄 Téléchargement d'un échantillon ({num_classes} classes, {images_per_class} images/classe)...")
        
        # Charger les noms de classes
        class_names = self.download_imagenet_labels()
        
        sample_dir = Path("imagenet_sample")
        sample_dir.mkdir(exist_ok=True)
        
        # Prendre les premières classes
        selected_classes = class_names[:num_classes]
        
        for i, class_name in enumerate(tqdm(selected_classes, desc="Classes")):
            class_dir = sample_dir / f"{i:03d}_{class_name.replace(' ', '_')}"
            class_dir.mkdir(exist_ok=True)
            
            # Simuler le téléchargement (placeholder)
            # Dans un vrai script, on utiliserait Flickr API ou autre source
            print(f"Classe {i+1}/{num_classes}: {class_name}")
            
            # Ici vous pourriez intégrer une vraie API de téléchargement d'images
            # Pour l'instant, on crée juste la structure
            
        print(f"✓ Structure d'échantillon créée dans {sample_dir}")

    def verify_dataset(self, dataset_path):
        """Vérifie l'intégrité du dataset téléchargé"""
        print("🔍 Vérification du dataset...")
        
        dataset_path = Path(dataset_path)
        if not dataset_path.exists():
            print(f"❌ Le dossier {dataset_path} n'existe pas")
            return False
        
        # Compter les classes et images
        class_dirs = [d for d in dataset_path.iterdir() if d.is_dir()]
        total_images = 0
        class_info = []
        
        for class_dir in class_dirs:
            images = list(class_dir.glob("*.JPEG")) + list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png"))
            total_images += len(images)
            class_info.append({
                "class": class_dir.name,
                "images": len(images)
            })
        
        print(f"📊 Statistiques:")
        print(f"   Classes: {len(class_dirs)}")
        print(f"   Images totales: {total_images}")
        print(f"   Moyenne par classe: {total_images/len(class_dirs):.1f}")
        
        # Afficher quelques exemples
        print(f"📋 Exemples de classes:")
        for info in class_info[:5]:
            print(f"   {info['class']}: {info['images']} images")
        
        return True

    def create_dataset_info(self):
        """Crée un fichier d'informations sur le dataset"""
        info = {
            "dataset_name": "ImageNet ILSVRC2012",
            "description": "Dataset complet ImageNet avec toutes les images par classe",
            "num_classes": 1000,
            "total_train_images": "~1,281,167",
            "total_val_images": 50000,
            "images_per_val_class": 50,
            "avg_images_per_train_class": "~1,281",
            "image_format": "JPEG",
            "organization": {
                "train": "train/n{wnid}/{image}.JPEG", 
                "val": "val/n{wnid}/{image}.JPEG"
            },
            "source": "http://www.image-net.org/",
            "paper": "ImageNet: A Large-Scale Hierarchical Image Database (Deng et al., 2009)",
            "download_methods": [
                "Site officiel (nécessite inscription)",
                "Academic Torrents (recommandé)",
                "Kaggle (simplifié)"
            ]
        }
        
        with open("imagenet_full_info.json", "w") as f:
            json.dump(info, f, indent=2)
        
        print("✓ Informations sauvegardées dans imagenet_full_info.json")

def main():
    """Fonction principale avec menu interactif"""
    downloader = ImageNetDownloader()
    
    print("🖼️  === TÉLÉCHARGEUR IMAGENET COMPLET ===\n")
    print("ImageNet complet contient:")
    print("- Training: ~1.28M images (1000+ images/classe)")
    print("- Validation: 50K images (50 images/classe)")
    print("- Taille totale: ~150 GB")
    print()
    
    while True:
        print("Options de téléchargement:")
        print("1. 🎯 Télécharger via Kaggle (RECOMMANDÉ)")
        print("2. 🌐 Télécharger depuis le site officiel") 
        print("3. 🔗 Créer script de téléchargement torrent")
        print("4. 📝 Télécharger échantillon (test)")
        print("5. 🔍 Vérifier dataset existant")
        print("6. ℹ️  Créer fichier d'informations")
        print("7. ❌ Quitter")
        
        choice = input("\nChoisissez une option (1-7): ").strip()
        
        if choice == "1":
            print("\n=== TÉLÉCHARGEMENT VIA KAGGLE ===")
            print("Prérequis:")
            print("1. pip install kaggle")
            print("2. Configurer ~/.kaggle/kaggle.json avec votre API key")
            print("3. Accepter les règles du dataset sur Kaggle")
            
            if input("Continuer ? (y/n): ").lower() == 'y':
                downloader.download_via_kaggle()
        
        elif choice == "2":
            print("\n=== TÉLÉCHARGEMENT OFFICIEL ===")
            subset = input("Quel subset ? (train/val/test): ").lower()
            if subset in ["train", "val", "test"]:
                if downloader.download_official_imagenet(subset):
                    # Organiser selon le subset
                    tar_file = f"ILSVRC2012_img_{subset}.tar"
                    if subset == "train":
                        downloader.extract_and_organize_train_set(tar_file)
                    elif subset == "val":
                        downloader.extract_and_organize_val_set(tar_file)
        
        elif choice == "3":
            print("\n=== CRÉATION SCRIPT TORRENT ===")
            downloader.create_alternative_download_script()
            print("Exécutez: ./download_imagenet_torrents.sh")
        
        elif choice == "4":
            print("\n=== ÉCHANTILLON DE TEST ===")
            num_classes = int(input("Nombre de classes (défaut 10): ") or "10")
            images_per_class = int(input("Images par classe (défaut 20): ") or "20")
            downloader.download_sample_dataset(num_classes, images_per_class)
        
        elif choice == "5":
            print("\n=== VÉRIFICATION DATASET ===")
            path = input("Chemin du dataset: ").strip() or "imagenet_full"
            downloader.verify_dataset(path)
        
        elif choice == "6":
            print("\n=== CRÉATION INFOS ===")
            downloader.create_dataset_info()
        
        elif choice == "7":
            print("Au revoir!")
            break
        
        else:
            print("Option invalide!")
        
        print("\n" + "="*50 + "\n")

if __name__ == "__main__":
    main()

🖼️  === TÉLÉCHARGEUR IMAGENET COMPLET ===

ImageNet complet contient:
- Training: ~1.28M images (1000+ images/classe)
- Validation: 50K images (50 images/classe)
- Taille totale: ~150 GB

Options de téléchargement:
1. 🎯 Télécharger via Kaggle (RECOMMANDÉ)
2. 🌐 Télécharger depuis le site officiel
3. 🔗 Créer script de téléchargement torrent
4. 📝 Télécharger échantillon (test)
5. 🔍 Vérifier dataset existant
6. ℹ️  Créer fichier d'informations
7. ❌ Quitter

=== TÉLÉCHARGEMENT VIA KAGGLE ===
Prérequis:
1. pip install kaggle
2. Configurer ~/.kaggle/kaggle.json avec votre API key
3. Accepter les règles du dataset sur Kaggle
🔄 Téléchargement via Kaggle...
Dataset URL: https://www.kaggle.com/datasets/c/imagenet-object-localization-challenge
❌ Erreur Kaggle: 403 Client Error: Forbidden for url: https://www.kaggle.com/api/v1/datasets/download/c/imagenet-object-localization-challenge?raw=false


Options de téléchargement:
1. 🎯 Télécharger via Kaggle (RECOMMANDÉ)
2. 🌐 Télécharger depuis le site off